In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from sklearn.linear_model import LogisticRegression, LinearRegression

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score

from sklearn.metrics import r2_score, mean_squared_error

from scipy.stats import ttest_1samp, ttest_rel, ttest_ind, f_oneway, shapiro, levene, bartlett, chisquare, chi2_contingency, kruskal

#### 제3유형

검정 방법

1. 분산분석(ANOVA) : A집단 vs B집단 vs C집단 ....
- (정규성O) ANOVA 분석
- (정규성X) 크루스칼-왈리스 검정(kruskal)

가설검정 순서

1. 가설 검정
2. 유의수준 확인
3. 정규성 검정 (주의) 집단 모두 정규성을 따라야한다!!
4. 등분산 검정
5. 검정 실시(통계량, p-value 확인)  (주의) 등분산 여부 확인   (f_oneway는 등분산을 가정하고 적용)
6. 귀무가설 기각여부 결정 (채택/기각)

##### 예제 문제

문제1-1
다음은 A,B,C 그룹 인원 성적 데이터이다.

세 그룹의 성적 평균이 같다고 할 수 있는지 ANOVA 분석을 실시하시오.

유의수준 5%

- A, B, C : 각 그룹 인원의 성적
- 귀무가설(H0) : 세 그룹의 성적 평균이 같다
- 대립가설(H1) : 세 그룹의 성적 평균이 같지 않다

In [4]:
# 데이터 만들기
df = pd.DataFrame( {
    'A': [120, 135, 122, 124, 135, 122, 145, 160, 155, 142, 144, 135, 167],
    'B' : [110, 132, 123, 119, 123, 115, 140, 162, 142, 138, 135, 142, 160],
    'C' : [130, 120, 115, 122, 133, 144, 122, 120, 110, 134, 125, 122, 122]})

df.head()

,A,B,C
0,120,110,130
1,135,132,120
2,122,123,115
3,124,119,122
4,135,123,133


In [6]:
# 각각 정규성 확인

a_statistic, a_pvalue = shapiro(df['A'])
b_statistic, b_pvalue = shapiro(df['B'])
c_statistic, c_pvalue = shapiro(df['C'])

print(a_pvalue, b_pvalue, c_pvalue)

if a_pvalue > 0.05 and b_pvalue > 0.05 and c_pvalue > 0.05 :
  print("A, B, C 모두 정규성을 만족한다")
else :
  print("A, B, C 중 1개 이상 정규성을 만족하지 않는다")

0.35585272312164307 0.5955665707588196 0.45265132188796997
A, B, C 모두 정규성을 만족한다


In [7]:
# 등분산성 확인

eq_statistic, eq_pvalue = bartlett(df['A'],df['B'],df['C'])

print(eq_pvalue)

0.12110174433684852


In [10]:
# 정규성, 등분산성을 만족하는 경우 ANOVA (f_oneway) 검정 실시

statistic, pvalue = f_oneway(df['A'],df['B'],df['C'])

print(pvalue)

if pvalue > 0.05 :
  print("A, B, C 그룹 성적 평균이 같다")
else :
  print("A, B, C 그룹 성적 평균이 같지 않다")

0.0346491232068625
A, B, C 그룹 성적 평균이 같지 않다


In [12]:
# 만약 정규성을 따르지 않는 경우 (kruskal)

x_statistic, x_pvalue = kruskal(df['A'],df['B'],df['C'])

print(x_pvalue)
if x_pvalue > 0.05 :
  print("A, B, C 그룹 성적 평균이 같다")
else :
  print("A, B, C 그룹 성적 평균이 같지 않다")

0.03179333314266727
A, B, C 그룹 성적 평균이 같지 않다


In [13]:
# 등분산성을 만족하지 않는 경우 Welch-ANOVA 분석
# import pingouin as pg   # 현재 시험 환경에서는 미지원!
# pg.welch_anova(dv='그룹변수명', between='성적데이터', data=데이터)
# pg.welch_anova(df['A'],df['B'],df['C']) 형태로 분석 불가

문제1-2 데이터 형태가 다른경우

In [14]:
# 데이터 만들기
df2 = pd.DataFrame( {
    '항목': ['A','A','A','A','A','A','A','A','A','A','A','A','A',
           'B','B','B','B','B','B','B','B','B','B','B','B','B',
           'C','C','C','C','C','C','C','C','C','C','C','C','C',],
    'value': [120, 135, 122, 124, 135, 122, 145, 160, 155, 142, 144, 135, 167,
             110, 132, 123, 119, 123, 115, 140, 162, 142, 138, 135, 142, 160,
             130, 120, 115, 122, 133, 144, 122, 120, 110, 134, 125, 122, 122]
    })

In [18]:
# 데이터가 합쳐져 있는 경우 각각 케이스별로 분류하여 진행하여야 한다
df_a = df2[df2['항목']=='A']['value']
df_b = df2[df2['항목']=='B']['value']
df_c = df2[df2['항목']=='C']['value']

print(df_a.shape, df_b.shape, df_c.shape)

(13,) (13,) (13,)


In [20]:
# 정규성 확인

a_statistic, a_pvalue = shapiro(df_a)
b_statistic, b_pvalue = shapiro(df_b)
c_statistic, c_pvalue = shapiro(df_c)

print(a_pvalue, b_pvalue, c_pvalue)

0.35585272312164307 0.5955665707588196 0.45265132188796997


In [21]:
# 등분산성 확인

eq_statistic, eq_pvalue = bartlett(df_a, df_b, df_c)

print(eq_pvalue)

0.12110174433684852


In [22]:
# 정규성, 등분산성을 만족하는 경우 ANOVA (f_oneway)

statistic, pvalue = f_oneway(df_a, df_b, df_c)

print(pvalue)
if pvalue > 0.05 :
  print('A, B, C 그룹 성적 평균이 같다')
else :
  print('A, B, C 그룹 성적 평균이 같지 않다')

0.0346491232068625
A, B, C 그룹 성적 평균이 같지 않다
